In [3]:
import pandas as pd
import numpy as np
from scipy import stats
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_recall_curve
from sklearn.metrics import recall_score, classification_report, auc, roc_curve
from sklearn.metrics import precision_recall_fscore_support, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from pylab import rcParams
from keras.models import Model, load_model
#from keras.layers import Input, Dense
from tensorflow.keras.layers import Input, Dense
from tensorflow import keras
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers

Using TensorFlow backend.


In [11]:
df = pd.read_csv('./DataP2/finaldata.csv')

In [12]:
print(df.shape)
df.head()

(9622407, 96)


,Unnamed: 0,date,serial_number,model,capacity_bytes,failure,smart_1_normalized,smart_1_raw,smart_2_normalized,smart_2_raw,...,smart_250_normalized,smart_250_raw,smart_251_normalized,smart_251_raw,smart_252_normalized,smart_252_raw,smart_254_normalized,smart_254_raw,smart_255_normalized,smart_255_raw
0,0,2016-01-01,MJ0351YNG9Z0XA,Hitachi HDS5C3030ALA630,1.482490e-311,0,100,0,135.0,108.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2016-01-01,Z305B2QN,ST4000DM000,1.976651e-311,0,113,54551400,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2016-01-01,MJ0351YNG9Z7LA,Hitachi HDS5C3030ALA630,1.482490e-311,0,100,0,136.0,104.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2016-01-01,MJ0351YNGABYAA,Hitachi HDS5C3030ALA630,1.482490e-311,0,100,0,136.0,104.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2016-01-01,WD-WMC4N2899475,WDC WD30EFRX,1.482490e-311,0,200,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
"""le = LabelEncoder()
encoded = le.fit_transform(df['model'])
le.inverse_transform(encoded)"""

array(['Hitachi HDS5C3030ALA630', 'Hitachi HDS5C3030ALA630',
       'Hitachi HDS5C4040ALE630', ..., 'Hitachi HDS5C3030ALA630',
       'ST4000DM000', 'ST4000DM000'], dtype=object)

In [13]:
#df.mean()
df.columns

Index(['Unnamed: 0', 'date', 'serial_number', 'model', 'capacity_bytes',
       'failure', 'smart_1_normalized', 'smart_1_raw', 'smart_2_normalized',
       'smart_2_raw', 'smart_3_normalized', 'smart_3_raw',
       'smart_4_normalized', 'smart_4_raw', 'smart_5_normalized',
       'smart_5_raw', 'smart_7_normalized', 'smart_7_raw',
       'smart_8_normalized', 'smart_8_raw', 'smart_9_normalized',
       'smart_9_raw', 'smart_10_normalized', 'smart_10_raw',
       'smart_11_normalized', 'smart_11_raw', 'smart_12_normalized',
       'smart_12_raw', 'smart_13_normalized', 'smart_13_raw',
       'smart_15_normalized', 'smart_15_raw', 'smart_22_normalized',
       'smart_22_raw', 'smart_183_normalized', 'smart_183_raw',
       'smart_184_normalized', 'smart_184_raw', 'smart_187_normalized',
       'smart_187_raw', 'smart_188_normalized', 'smart_188_raw',
       'smart_189_normalized', 'smart_189_raw', 'smart_190_normalized',
       'smart_190_raw', 'smart_191_normalized', 'smart_191_raw',
 

In [14]:
# Constants
RANDOM_SEED = 314
TEST_PCT = 0.2

In [ ]:
df.apply(pd.value_counts)

In [6]:
df_norm = df

# Drop the following columns
df_norm.drop(['date', 'model', 'serial_number', 'Unnamed: 0'], axis=1, inplace=True)

# Drop normalized columns
df_norm = df_norm[df_norm.columns.drop(list(df_norm.filter(regex='normalized')))]

# Drop Duplicate Rows
df_norm = df_norm.drop_duplicates(subset=None, keep='first', inplace=False).reset_index(drop=True)

# Drop Columns where all entries are NAN
df_norm.dropna(axis=1, how='all', inplace=True)

# Replace NaNs with mean
df_norm.fillna(df_norm.mean(), inplace=True)

display(df_norm.head())
df_norm.shape

,capacity_bytes,failure,smart_1_raw,smart_2_raw,smart_3_raw,smart_4_raw,smart_5_raw,smart_7_raw,smart_8_raw,smart_9_raw,...,smart_224_raw,smart_225_raw,smart_226_raw,smart_240_raw,smart_241_raw,smart_242_raw,smart_250_raw,smart_251_raw,smart_252_raw,smart_254_raw
0,1.482490e-311,0,0,108.000000,540,14,0,0.000000e+00,37.000000,27929,...,0.0,125077.414532,348.76827,0.000000e+00,0.000000e+00,0.000000e+00,7.817648e+07,1.428992e+08,6.556180e+07,0.0
1,1.976651e-311,0,54551400,81.604777,0,4,0,1.279800e-317,32.143245,411,...,0.0,125077.414532,348.76827,1.319155e-321,3.467963e-315,1.316882e-315,7.817648e+07,1.428992e+08,6.556180e+07,0.0
2,1.482490e-311,0,0,104.000000,566,19,0,0.000000e+00,37.000000,27492,...,0.0,125077.414532,348.76827,0.000000e+00,0.000000e+00,0.000000e+00,7.817648e+07,1.428992e+08,6.556180e+07,0.0
3,1.482490e-311,0,0,104.000000,507,15,0,0.000000e+00,37.000000,26640,...,0.0,125077.414532,348.76827,0.000000e+00,0.000000e+00,0.000000e+00,7.817648e+07,1.428992e+08,6.556180e+07,0.0
4,1.482490e-311,0,0,81.604777,6250,10,0,0.000000e+00,32.143245,12198,...,0.0,125077.414532,348.76827,0.000000e+00,0.000000e+00,0.000000e+00,7.817648e+07,1.428992e+08,6.556180e+07,0.0


(9377465, 45)

In [7]:
# Split
train_x, test_x = train_test_split(df_norm, test_size=TEST_PCT, random_state=RANDOM_SEED)

# Transform to DF
trainDf = pd.DataFrame(train_x)
trainDf.columns = df_norm.columns

testDf = pd.DataFrame(test_x)
testDf.columns = df_norm.columns

# Training Data
train_x = trainDf[trainDf.failure == 0] # Healthy drives
train_x = trainDf.drop(['failure'], axis=1) # drop the Failure column

# Testing Data
test_y = testDf['failure']
test_x = testDf.drop(['failure'], axis=1) #drop the failure column

display(train_x.head())
display(test_x.head())
display(test_y.head())

# Scale and Standardize & Transform DF to ndarray
train_x = StandardScaler().fit_transform(train_x)
test_x = StandardScaler().fit_transform(test_x)
test_y = test_y.values

print(train_x)
print(test_x)

,capacity_bytes,smart_1_raw,smart_2_raw,smart_3_raw,smart_4_raw,smart_5_raw,smart_7_raw,smart_8_raw,smart_9_raw,smart_10_raw,...,smart_224_raw,smart_225_raw,smart_226_raw,smart_240_raw,smart_241_raw,smart_242_raw,smart_250_raw,smart_251_raw,smart_252_raw,smart_254_raw
1759233,1.976651e-311,208786832,81.604777,0,1,0,1.221463e-315,32.143245,6782,0,...,0.0,125077.414532,348.76827,3.214391e-320,1.212084e-313,3.273992e-314,7.817648e+07,1.428992e+08,6.556180e+07,0.0
1272381,1.976651e-311,153923480,81.604777,0,2,0,1.315462e-315,32.143245,7737,0,...,0.0,125077.414532,348.76827,3.754405e-320,1.285532e-313,2.116224e-314,7.817648e+07,1.428992e+08,6.556180e+07,0.0
9345586,1.976651e-311,83539592,81.604777,0,3,0,3.915898e+08,32.143245,9439,0,...,0.0,125077.414532,348.76827,9.188000e+03,3.266810e+10,6.894061e+09,7.817648e+07,1.428992e+08,6.556180e+07,0.0
1917615,1.976651e-311,61243168,81.604777,0,2,0,6.777619e-316,32.143245,3821,0,...,0.0,125077.414532,348.76827,1.817668e-320,7.758671e-314,8.041525e-315,7.817648e+07,1.428992e+08,6.556180e+07,0.0
6466407,1.976651e-311,0,108.000000,0,4,0,0.000000e+00,43.000000,19638,0,...,0.0,125077.414532,348.76827,1.099766e+13,1.342085e+10,4.409590e+10,7.817648e+07,1.428992e+08,6.556180e+07,0.0


,capacity_bytes,smart_1_raw,smart_2_raw,smart_3_raw,smart_4_raw,smart_5_raw,smart_7_raw,smart_8_raw,smart_9_raw,smart_10_raw,...,smart_224_raw,smart_225_raw,smart_226_raw,smart_240_raw,smart_241_raw,smart_242_raw,smart_250_raw,smart_251_raw,smart_252_raw,smart_254_raw
256338,1.976651e-311,14588680,81.604777,0,15,0,3.512602e-315,32.143245,10707,0,...,0.0,125077.414532,348.76827,5.245495e-320,9.781328e-314,9.557874e-313,7.817648e+07,1.428992e+08,6.556180e+07,0.0
8715101,1.976651e-311,134381384,81.604777,0,4,0,4.388257e+08,32.143245,11863,0,...,0.0,125077.414532,348.76827,1.171600e+04,3.559665e+10,8.307753e+09,7.817648e+07,1.428992e+08,6.556180e+07,0.0
7637951,1.482490e-311,0,101.000000,555,13,212,0.000000e+00,37.000000,32188,0,...,0.0,125077.414532,348.76827,1.099766e+13,1.342085e+10,4.409590e+10,7.817648e+07,1.428992e+08,6.556180e+07,0.0
1108539,1.482490e-311,0,0.000000,565,27,0,0.000000e+00,0.000000,38242,0,...,0.0,125077.414532,348.76827,0.000000e+00,0.000000e+00,0.000000e+00,7.817648e+07,1.428992e+08,6.556180e+07,0.0
4943692,1.976651e-311,0,108.000000,554,12,0,0.000000e+00,42.000000,17453,0,...,0.0,125077.414532,348.76827,1.099766e+13,1.342085e+10,4.409590e+10,7.817648e+07,1.428992e+08,6.556180e+07,0.0


256338     0
8715101    0
7637951    0
1108539    0
4943692    0
Name: failure, dtype: int64

[[ 1.42838893e-313  1.71126431e+000  2.36835874e-004 ...  2.11698131e-004
   2.11754353e-004  0.00000000e+000]
 [ 1.42838893e-313  1.02957856e+000  2.36835874e-004 ...  2.11698131e-004
   2.11754353e-004  0.00000000e+000]
 [ 1.42838893e-313  1.55047709e-001  2.36835874e-004 ...  2.11698131e-004
   2.11754353e-004  0.00000000e+000]
 ...
 [ 1.42838893e-313 -8.82944825e-001  2.36835874e-004 ...  2.11698131e-004
   2.11754353e-004  0.00000000e+000]
 [-9.74039147e-312 -8.82944825e-001 -3.20345797e+000 ...  2.11698131e-004
   2.11754353e-004  0.00000000e+000]
 [ 1.42838893e-313 -6.32883872e-001  2.36835874e-004 ...  2.11698131e-004
   2.11754353e-004  0.00000000e+000]]
[[ 1.35695393e-313 -7.01885620e-001 -9.49113850e-004 ... -8.61797204e-004
  -8.61844298e-004  0.00000000e+000]
 [ 1.35695393e-313  7.86260352e-001 -9.49113850e-004 ... -8.61797204e-004
  -8.61844298e-004  0.00000000e+000]
 [-4.80591979e-312 -8.83116068e-001  7.61904410e-001 ... -8.61797204e-004
  -8.61844298e-004  0.00000000e+

In [8]:
train_x.shape[1]

44

In [9]:
# AutoEncoder

# Parameters
nb_epoch = 2000
batch_size = 50000
# Network 44 -> 32 -> 16 -> 8    -> 16 -> 32 -> 44
input_dim = train_x.shape[1] #num of columns 44
encoding_dim = 32 
hidden_dim1 = 16
hidden_dim2 = 8
learning_rate = 1e-7

# Encoder
input_layer = keras.Input(shape=(input_dim, ))
encoder = keras.Dense(encoding_dim, activation="relu", activity_regularizer=regularizers.l1(learning_rate))(input_layer)
encoder = keras.Dense(hidden_dim1, activation="relu")(encoder)
encoder = keras.Dense(hidden_dim2, activation="relu")(encoder) #

# Decoder
decoder = keras.Dense(hidden_dim2, activation='relu')(encoder)
decoder = keras.Dense(hidden_dim1, activation='relu')(decoder) #
decoder = keras.Dense(input_dim, activation='relu')(decoder) 

# Model
autoencoder = keras.Model(inputs=input_layer, outputs=decoder)

In [10]:
# Train Model

autoencoder.compile(metrics=['accuracy'],
                    loss='mean_squared_error',
                    optimizer='adam')

#cp = keras.callbacks.ModelCheckpoint(filepath="autoencoder.h5",
#                               save_best_only=True,
#                               verbose=0)
#logdir = os.path.join('logs')
#tb = keras.callbacks.TensorBoard(log_dir=logdir,
#                histogram_freq=0,
#                write_graph=True,
#                write_images=True)

history = autoencoder.fit(train_x, train_x,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    shuffle=True,
                    validation_data=(test_x, test_x),
                    verbose=1, #callbacks=[cp, tb]
                    ).history

Train on 7501972 samples, validate on 1875493 samples
Epoch 1/2000
7501972/7501972 [==============================] - 12s 2us/sample - loss: 0.8309 - accuracy: 0.2637 - val_loss: 0.7525 - val_accuracy: 0.4157
Epoch 2/2000
7501972/7501972 [==============================] - 7s 1us/sample - loss: 0.6420 - accuracy: 0.5078 - val_loss: 0.5439 - val_accuracy: 0.5922
Epoch 3/2000
7501972/7501972 [==============================] - 7s 1us/sample - loss: 0.4868 - accuracy: 0.6229 - val_loss: 0.4559 - val_accuracy: 0.6464
Epoch 4/2000
7501972/7501972 [==============================] - 7s 1us/sample - loss: 0.4430 - accuracy: 0.6693 - val_loss: 0.4333 - val_accuracy: 0.6867
Epoch 5/2000
7501972/7501972 [==============================] - 7s 1us/sample - loss: 0.4333 - accuracy: 0.6944 - val_loss: 0.4304 - val_accuracy: 0.6996
Epoch 6/2000
7501972/7501972 [==============================] - 7s 1us/sample - loss: 0.4317 - accuracy: 0.7044 - val_loss: 0.4296 - val_accuracy: 0.7062
Epoch 7/2000
7501972/

Epoch 106/2000
7501972/7501972 [==============================] - 7s 1us/sample - loss: 0.4255 - accuracy: 0.7513 - val_loss: 0.4232 - val_accuracy: 0.7506
Epoch 107/2000
7501972/7501972 [==============================] - 7s 1us/sample - loss: 0.4255 - accuracy: 0.7513 - val_loss: 0.4232 - val_accuracy: 0.7506
Epoch 108/2000
7501972/7501972 [==============================] - 7s 1us/sample - loss: 0.4255 - accuracy: 0.7518 - val_loss: 0.4232 - val_accuracy: 0.7509
Epoch 109/2000
7501972/7501972 [==============================] - 7s 1us/sample - loss: 0.4255 - accuracy: 0.7515 - val_loss: 0.4232 - val_accuracy: 0.7471
Epoch 110/2000
7501972/7501972 [==============================] - 6s 1us/sample - loss: 0.4255 - accuracy: 0.7515 - val_loss: 0.4232 - val_accuracy: 0.7508
Epoch 111/2000
7501972/7501972 [==============================] - 7s 1us/sample - loss: 0.4255 - accuracy: 0.7513 - val_loss: 0.4232 - val_accuracy: 0.7486
Epoch 112/2000
7501972/7501972 [==============================] 

7501972/7501972 [==============================] - 6s 1us/sample - loss: 0.4253 - accuracy: 0.7535 - val_loss: 0.4230 - val_accuracy: 0.7511
Epoch 211/2000
7501972/7501972 [==============================] - 7s 1us/sample - loss: 0.4253 - accuracy: 0.7531 - val_loss: 0.4230 - val_accuracy: 0.7494
Epoch 212/2000
7501972/7501972 [==============================] - 7s 1us/sample - loss: 0.4253 - accuracy: 0.7527 - val_loss: 0.4230 - val_accuracy: 0.7526
Epoch 213/2000
7501972/7501972 [==============================] - 7s 1us/sample - loss: 0.4253 - accuracy: 0.7533 - val_loss: 0.4230 - val_accuracy: 0.7534
Epoch 214/2000
7501972/7501972 [==============================] - 7s 1us/sample - loss: 0.4253 - accuracy: 0.7536 - val_loss: 0.4230 - val_accuracy: 0.7503
Epoch 215/2000
7501972/7501972 [==============================] - 6s 1us/sample - loss: 0.4253 - accuracy: 0.7526 - val_loss: 0.4232 - val_accuracy: 0.7475
Epoch 216/2000
7501972/7501972 [==============================] - 7s 1us/sample

7501972/7501972 [==============================] - 7s 1us/sample - loss: 0.4251 - accuracy: 0.7549 - val_loss: 0.4229 - val_accuracy: 0.7519
Epoch 315/2000
7501972/7501972 [==============================] - 7s 1us/sample - loss: 0.4252 - accuracy: 0.7530 - val_loss: 0.4230 - val_accuracy: 0.7504
Epoch 316/2000
7501972/7501972 [==============================] - 7s 1us/sample - loss: 0.4252 - accuracy: 0.7539 - val_loss: 0.4228 - val_accuracy: 0.7533
Epoch 317/2000
7501972/7501972 [==============================] - 6s 1us/sample - loss: 0.4253 - accuracy: 0.7519 - val_loss: 0.4230 - val_accuracy: 0.7500
Epoch 318/2000
7501972/7501972 [==============================] - 6s 1us/sample - loss: 0.4252 - accuracy: 0.7538 - val_loss: 0.4228 - val_accuracy: 0.7538
Epoch 319/2000
7501972/7501972 [==============================] - 7s 1us/sample - loss: 0.4251 - accuracy: 0.7549 - val_loss: 0.4228 - val_accuracy: 0.7534
Epoch 320/2000
7501972/7501972 [==============================] - 7s 1us/sample

7501972/7501972 [==============================] - 7s 1us/sample - loss: 0.4251 - accuracy: 0.7537 - val_loss: 0.4231 - val_accuracy: 0.7525
Epoch 419/2000
7501972/7501972 [==============================] - 7s 1us/sample - loss: 0.4251 - accuracy: 0.7541 - val_loss: 0.4231 - val_accuracy: 0.7510
Epoch 420/2000
7501972/7501972 [==============================] - 7s 1us/sample - loss: 0.4251 - accuracy: 0.7544 - val_loss: 0.4231 - val_accuracy: 0.7538
Epoch 421/2000
7501972/7501972 [==============================] - 7s 1us/sample - loss: 0.4251 - accuracy: 0.7542 - val_loss: 0.4230 - val_accuracy: 0.7535
Epoch 422/2000
7501972/7501972 [==============================] - 7s 1us/sample - loss: 0.4251 - accuracy: 0.7539 - val_loss: 0.4230 - val_accuracy: 0.7529
Epoch 423/2000
7501972/7501972 [==============================] - 7s 1us/sample - loss: 0.4251 - accuracy: 0.7536 - val_loss: 0.4230 - val_accuracy: 0.7539
Epoch 424/2000
7501972/7501972 [==============================] - 7s 1us/sample

...

7501972/7501972 [==============================] - 7s 1us/sample - loss: 0.4014 - accuracy: 0.7581 - val_loss: 0.3991 - val_accuracy: 0.7580
Epoch 1563/2000
7501972/7501972 [==============================] - 7s 1us/sample - loss: 0.4014 - accuracy: 0.7582 - val_loss: 0.3995 - val_accuracy: 0.7524
Epoch 1564/2000
7501972/7501972 [==============================] - 7s 1us/sample - loss: 0.4014 - accuracy: 0.7569 - val_loss: 0.3991 - val_accuracy: 0.7581
Epoch 1565/2000
7501972/7501972 [==============================] - 7s 1us/sample - loss: 0.4017 - accuracy: 0.7546 - val_loss: 0.3996 - val_accuracy: 0.7495
Epoch 1566/2000
7501972/7501972 [==============================] - 7s 1us/sample - loss: 0.4023 - accuracy: 0.7539 - val_loss: 0.3998 - val_accuracy: 0.7554
Epoch 1567/2000
7501972/7501972 [==============================] - 7s 1us/sample - loss: 0.4015 - accuracy: 0.7584 - val_loss: 0.3992 - val_accuracy: 0.7560
Epoch 1568/2000
7501972/7501972 [==============================] - 7s 1us/

7501972/7501972 [==============================] - 7s 1us/sample - loss: 0.4015 - accuracy: 0.7560 - val_loss: 0.3994 - val_accuracy: 0.7550
Epoch 1667/2000
7501972/7501972 [==============================] - 6s 1us/sample - loss: 0.4015 - accuracy: 0.7568 - val_loss: 0.3993 - val_accuracy: 0.7559
Epoch 1668/2000
7501972/7501972 [==============================] - 7s 1us/sample - loss: 0.4015 - accuracy: 0.7576 - val_loss: 0.3993 - val_accuracy: 0.7568
Epoch 1669/2000
7501972/7501972 [==============================] - 7s 1us/sample - loss: 0.4014 - accuracy: 0.7581 - val_loss: 0.3993 - val_accuracy: 0.7569
Epoch 1670/2000
7501972/7501972 [==============================] - 7s 1us/sample - loss: 0.4014 - accuracy: 0.7585 - val_loss: 0.3993 - val_accuracy: 0.7565
Epoch 1671/2000
7501972/7501972 [==============================] - 7s 1us/sample - loss: 0.4014 - accuracy: 0.7587 - val_loss: 0.3993 - val_accuracy: 0.7572
Epoch 1672/2000
7501972/7501972 [==============================] - 7s 1us/

7501972/7501972 [==============================] - 7s 1us/sample - loss: 0.4016 - accuracy: 0.7548 - val_loss: 0.3992 - val_accuracy: 0.7571
Epoch 1771/2000
7501972/7501972 [==============================] - 7s 1us/sample - loss: 0.4014 - accuracy: 0.7586 - val_loss: 0.3991 - val_accuracy: 0.7577
Epoch 1772/2000
7501972/7501972 [==============================] - 7s 1us/sample - loss: 0.4013 - accuracy: 0.7588 - val_loss: 0.3991 - val_accuracy: 0.7574
Epoch 1773/2000
7501972/7501972 [==============================] - 7s 1us/sample - loss: 0.4013 - accuracy: 0.7585 - val_loss: 0.3992 - val_accuracy: 0.7556
Epoch 1774/2000
7501972/7501972 [==============================] - 7s 1us/sample - loss: 0.4014 - accuracy: 0.7577 - val_loss: 0.3991 - val_accuracy: 0.7577
Epoch 1775/2000
7501972/7501972 [==============================] - 7s 1us/sample - loss: 0.4013 - accuracy: 0.7587 - val_loss: 0.3991 - val_accuracy: 0.7582
Epoch 1776/2000
7501972/7501972 [==============================] - 7s 1us/

7501972/7501972 [==============================] - 7s 1us/sample - loss: 0.3941 - accuracy: 0.7596 - val_loss: 0.3922 - val_accuracy: 0.7575
Epoch 1875/2000
7501972/7501972 [==============================] - 7s 1us/sample - loss: 0.3940 - accuracy: 0.7599 - val_loss: 0.3921 - val_accuracy: 0.7584
Epoch 1876/2000
7501972/7501972 [==============================] - 7s 1us/sample - loss: 0.3941 - accuracy: 0.7598 - val_loss: 0.3922 - val_accuracy: 0.7582
Epoch 1877/2000
7501972/7501972 [==============================] - 7s 1us/sample - loss: 0.3941 - accuracy: 0.7593 - val_loss: 0.3922 - val_accuracy: 0.7587
Epoch 1878/2000
7501972/7501972 [==============================] - 7s 1us/sample - loss: 0.3941 - accuracy: 0.7597 - val_loss: 0.3922 - val_accuracy: 0.7589
Epoch 1879/2000
7501972/7501972 [==============================] - 7s 1us/sample - loss: 0.3940 - accuracy: 0.7598 - val_loss: 0.3922 - val_accuracy: 0.7569
Epoch 1880/2000
7501972/7501972 [==============================] - 7s 1us/

7501972/7501972 [==============================] - 7s 1us/sample - loss: 0.3940 - accuracy: 0.7596 - val_loss: 0.3921 - val_accuracy: 0.7576
Epoch 1979/2000
7501972/7501972 [==============================] - 7s 1us/sample - loss: 0.3941 - accuracy: 0.7587 - val_loss: 0.3922 - val_accuracy: 0.7553
Epoch 1980/2000
7501972/7501972 [==============================] - 7s 1us/sample - loss: 0.3941 - accuracy: 0.7588 - val_loss: 0.3922 - val_accuracy: 0.7584
Epoch 1981/2000
7501972/7501972 [==============================] - 7s 1us/sample - loss: 0.3940 - accuracy: 0.7593 - val_loss: 0.3922 - val_accuracy: 0.7569
Epoch 1982/2000
7501972/7501972 [==============================] - 7s 1us/sample - loss: 0.3940 - accuracy: 0.7593 - val_loss: 0.3923 - val_accuracy: 0.7531
Epoch 1983/2000
7501972/7501972 [==============================] - 7s 1us/sample - loss: 0.3941 - accuracy: 0.7581 - val_loss: 0.3922 - val_accuracy: 0.7574
Epoch 1984/2000
7501972/7501972 [==============================] - 7s 1us/

In [11]:
# serialize model to JSON
model_json = autoencoder.to_json()
with open("autoencoderModel1.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
autoencoder.save_weights("autoencoder1.h5")
print("Saved model to disk")

Saved model to disk


In [4]:
# load json and create model
json_file = open('autoencoderModel1.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = keras.models.model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("autoencoder1.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

pred = loaded_model.predict(test_x)
mse = np.mean(np.power(test_x - pred, 2), axis=1)
error_df = pd.DataFrame({'Reconstruction_error': mse,
                        'True_class': test_y})

Loaded model from disk


NameError: name 'test_x' is not defined

In [ ]:
# Metrics
display(error_df.describe())
error_df

In [ ]:
error_df.iloc[np.where(test_y == 1)]

In [8]:
plt.plot(loaded_model.history['loss'])


AttributeError: 'Model' object has no attribute 'history'